<a href="https://colab.research.google.com/github/Ch256981/IMLP347/blob/main/HW1_Ko_I_Hung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BUDT 758J - HW1

I-Hung Ko

In [129]:
# install PySpark
!pip install -q pyspark

In [98]:
# create a SparkSession from pyspark.sql, which is the entry point to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HW1').getOrCreate()

In [99]:
# verify
spark

#Load Dataset

In [100]:
# 703
spark.read.json('hw1_customer.json').createOrReplaceTempView('Customer')
spark.read.json('hw1_order.json').createOrReplaceTempView('Order')
spark.read.json('hw1_orderline.json').createOrReplaceTempView('Orderline')
spark.read.json('hw1_product.json').createOrReplaceTempView('Product')

In [101]:
# 704
import pandas as pd
pd_customer = pd.read_json('hw1_customer.json')
pd_order = pd.read_json('hw1_order.json')
pd_orderline = pd.read_json('hw1_orderline.json')
pd_product = pd.read_json('hw1_product.json')

In [102]:
# 758J
sp_customer = spark.read.json('hw1_customer.json')
sp_order = spark.read.json('hw1_order.json')
sp_orderline = spark.read.json('hw1_orderline.json')
sp_product = spark.read.json('hw1_product.json')

#Data Manipulation

#1. How many customers in each of the state, New Jersey or New York?

In [89]:
# 703
spark.sql('''
SELECT c.customerState AS `Customer State`, COUNT(c.customerId) AS `Count of Customers`
FROM Customer c
WHERE customerState IN ('NJ', 'NY')
GROUP BY c.customerState
''').show()

+--------------+------------------+
|Customer State|Count of Customers|
+--------------+------------------+
|            NJ|                 2|
|            NY|                 1|
+--------------+------------------+



In [134]:
# 704
pd_customer.groupby('customerState')['customerId'].count().reset_index()\
   .query('customerState in [\'NJ\', \'NY\']')\
   .rename(columns={'customerState': 'Customer State', 'customerId': 'Count of Customers'})

,Customer State,Count of Customers
5,NJ,2
6,NY,1


In [135]:
pd_customer.drop_duplicates(subset=['customerState', 'customerId'])\
      .groupby('customerState').size()\
      .reset_index(name='Count of Customers')\
      .query("customerState in ['NJ', 'NY']")\
      .rename(columns={'customerState': 'Customer State'})

,Customer State,Count of Customers
5,NJ,2
6,NY,1


In [136]:
# 758J
from pyspark.sql.functions import col
sp_customer.filter('customerState IN (\'NJ\', \'NY\')').groupBy('customerState')\
   .count().select(col('customerState').alias('Customer State')\
           ,col('count').alias('Count of Customers')).show()

+--------------+------------------+
|Customer State|Count of Customers|
+--------------+------------------+
|            NJ|                 2|
|            NY|                 1|
+--------------+------------------+



#2. How many orders in each month?

In [93]:
# 703
spark.sql('''SELECT Month(o.orderDate) AS Month, COUNT (o.orderId) AS `Count of Orders`
      FROM Order o
      GROUP BY Month(orderDate)
      ORDER BY Month''').show()

+-----+---------------+
|Month|Count of Orders|
+-----+---------------+
|   10|              8|
|   11|              2|
+-----+---------------+



In [138]:
# 704
pd_order.groupby(pd.to_datetime(pd_order['orderDate'])\
   .dt.month)['orderId'].count()\
   .reset_index(name='Count of Orders')\
   .rename(columns={'orderDate': 'Month'})

,Month,Count of Orders
0,10,8
1,11,2


In [144]:
# 758J
from pyspark.sql.functions import month, count
sp_order.select(month('orderDate').alias('Month'))\
               .groupBy('Month')\
               .agg(count('*').alias('Count of Orders')).show()

+-----+---------------+
|Month|Count of Orders|
+-----+---------------+
|   10|              8|
|   11|              2|
+-----+---------------+



#3. What is the product id with total ordered quantity greater than 4 in the ascending order of product id?

In [116]:
# 703
spark.sql('''SELECT l.productId
      FROM Orderline l
      GROUP BY l.productId
      HAVING SUM(orderedQuantity) > 4
      ORDER BY l.productId ASC''').show()

+---------+
|productId|
+---------+
|        1|
|        3|
|        4|
|        7|
|        8|
+---------+



In [140]:
# 704
pd_orderline.groupby('productId').sum().reset_index().query('orderedQuantity > 4').sort_values('productId')[['productId']]

,productId
0,1
2,3
3,4
6,7
7,8


In [141]:
# 758J
sp_orderline.groupBy('productId').sum('orderedQuantity').filter('sum(orderedQuantity) > 4').select('productId').orderBy('productId').show()

+---------+
|productId|
+---------+
|        1|
|        3|
|        4|
|        7|
|        8|
+---------+



#4. What are the description, finish, and price for all products, if its price is lower than the average price for all products, in the descending order of price difference?

In [119]:
# 703
spark.sql('''SELECT p.productDescription, p.productFinish, p.productPrice,
      (SELECT AVG(productPrice) FROM Product) - p.productPrice AS `Price Difference`
      FROM Product p
      WHERE p.productPrice < (SELECT AVG(productPrice) FROM Product)
      ORDER BY `Price Difference` DESC''').show()

+--------------------+-------------+------------+------------------+
|  productDescription|productFinish|productPrice|  Price Difference|
+--------------------+-------------+------------+------------------+
|         48 Bookcase|       Walnut|         150|384.63157894736844|
|          Nightstand|       Cherry|         150|384.63157894736844|
|    Cherry End Table|       Cherry|         175|359.63157894736844|
|         48 Bookcase|          Oak|         175|359.63157894736844|
| Birch Coffee Tables|        Birch|         200|334.63157894736844|
|         96 Bookcase|          Oak|         200|334.63157894736844|
|         96 Bookcase|       Walnut|         225|309.63157894736844|
|      Pine End Table|         Pine|         256|278.63157894736844|
|       Writer's Desk|        Birch|         300|234.63157894736844|
|       Writer's Desk|          Oak|         325|209.63157894736844|
|High Back Leather...|      Leather|         362|172.63157894736844|
|    4-Drawer Dresser|          Oa

In [146]:
# 704
pd_product[pd_product['productPrice'] < pd_product['productPrice'].mean()]\
 .sort_values(by='productPrice', ascending=True)\
 .assign(Price_Difference=pd_product['productPrice'].mean() - pd_product['productPrice'])\
 [['productDescription', 'productFinish', 'productPrice', 'Price_Difference']]

,productDescription,productFinish,productPrice,Price_Difference
6,48 Bookcase,Walnut,150,384.631579
12,Nightstand,Cherry,150,384.631579
0,Cherry End Table,Cherry,175,359.631579
7,48 Bookcase,Oak,175,359.631579
1,Birch Coffee Tables,Birch,200,334.631579
9,96 Bookcase,Oak,200,334.631579
8,96 Bookcase,Walnut,225,309.631579
18,Pine End Table,Pine,256,278.631579
13,Writer's Desk,Birch,300,234.631579
4,Writer's Desk,Oak,325,209.631579


In [147]:
# 758J
from pyspark.sql.functions import avg, col

average_price = sp_product.select(avg('productPrice')).first()[0]

sp_product.where(sp_product['productPrice'] < average_price)\
   .withColumn('Price_Difference', average_price - col('productPrice'))\
   .orderBy(col('Price_Difference').desc())\
   .select('productDescription', 'productFinish', 'productPrice', 'Price_Difference').show()

+--------------------+-------------+------------+------------------+
|  productDescription|productFinish|productPrice|  Price_Difference|
+--------------------+-------------+------------+------------------+
|         48 Bookcase|       Walnut|         150|384.63157894736844|
|          Nightstand|       Cherry|         150|384.63157894736844|
|    Cherry End Table|       Cherry|         175|359.63157894736844|
|         48 Bookcase|          Oak|         175|359.63157894736844|
| Birch Coffee Tables|        Birch|         200|334.63157894736844|
|         96 Bookcase|          Oak|         200|334.63157894736844|
|         96 Bookcase|       Walnut|         225|309.63157894736844|
|      Pine End Table|         Pine|         256|278.63157894736844|
|       Writer's Desk|        Birch|         300|234.63157894736844|
|       Writer's Desk|          Oak|         325|209.63157894736844|
|High Back Leather...|      Leather|         362|172.63157894736844|
|    4-Drawer Dresser|          Oa